In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, precision_score, confusion_matrix, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer

from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [10]:
df= pd.read_csv('Data/diabetes_clean.csv')
df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80,No,Yes,never,25.19,6.6,140.0,0
1,Female,54,No,No,No Info,27.32,6.6,80.0,0
2,Male,28,No,No,never,27.32,5.7,158.0,0
3,Female,36,No,No,current,23.45,5.0,155.0,0
4,Male,76,Yes,Yes,current,20.14,4.8,155.0,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80,No,No,No Info,27.32,6.2,90.0,0
99996,Female,2,No,No,No Info,17.37,6.5,100.0,0
99997,Male,66,No,No,former,27.83,5.7,155.0,0
99998,Female,24,No,No,never,35.42,4.0,100.0,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  int64  
 2   hypertension         100000 non-null  object 
 3   heart_disease        100000 non-null  object 
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  float64
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 6.9+ MB


In [12]:
df['hypertension'] = df['hypertension'].astype('category')
df['heart_disease'] = df['heart_disease'].astype('category')

In [13]:
X=df.drop(columns="diabetes")
y=df['diabetes']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
trf1=ColumnTransformer(transformers=[
    ('encode',OneHotEncoder(drop='first',handle_unknown='ignore'),['gender','hypertension','heart_disease','smoking_history']),
    ('scale',MinMaxScaler(),['age','bmi','HbA1c_level','blood_glucose_level'])
],remainder='passthrough')

In [16]:
pipe1= Pipeline(steps=[
    ('trf1',trf1),
    ('smote',SMOTE(random_state=42)),
    ('lr',LogisticRegression())
])
pipe1.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encode',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['gender', 'hypertension',
                                                   'heart_disease',
                                                   'smoking_history']),
                                                 ('scale', MinMaxScaler(),
                                                  ['age', 'bmi', 'HbA1c_level',
                                                   'blood_glucose_level'])])),
                ('smote', SMOTE(random_state=42)),
                ('lr', LogisticRegression())])

In [17]:
y_pred1 =pipe1.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred1))
print("Precision:", precision_score(y_test, y_pred1))
print("Recall:", recall_score(y_test, y_pred1))
print("F1 Score:", f1_score(y_test, y_pred1))
print("ROC-AUC:", roc_auc_score(y_test, pipe1.predict_proba(X_test)[:, 1]))
report = classification_report(y_test, y_pred1)
print(report)

Accuracy: 0.84435
Precision: 0.3376473307141206
Recall: 0.8553864168618267
F1 Score: 0.48417564208782105
ROC-AUC: 0.9374423706041622
              precision    recall  f1-score   support

           0       0.98      0.84      0.91     18292
           1       0.34      0.86      0.48      1708

    accuracy                           0.84     20000
   macro avg       0.66      0.85      0.70     20000
weighted avg       0.93      0.84      0.87     20000



In [18]:
param_grid1= {
    'lr__C': [0.001, 0.01, 0.1, 1, 10],  
    'lr__penalty': ['l2'],                
    'lr__solver': ['liblinear', 'saga'],  
    'lr__max_iter': [100, 200, 300]       
}

In [19]:
grid_search1 = GridSearchCV(pipe1, param_grid=param_grid1, cv=5, scoring='recall', n_jobs=-1)
grid_search1.fit(X_train, y_train)
print("Best Parameters:", grid_search1.best_params_)
print("Best Score:", grid_search1.best_score_)

Best Parameters: {'lr__C': 0.001, 'lr__max_iter': 100, 'lr__penalty': 'l2', 'lr__solver': 'liblinear'}
Best Score: 0.8673446320336469


In [20]:
model1 = grid_search1.best_estimator_
y_pred1 = model1.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred1))
print("Precision:", precision_score(y_test, y_pred1))
print("Recall:", recall_score(y_test, y_pred1))
print("F1 Score:", f1_score(y_test, y_pred1))
print("ROC-AUC:", roc_auc_score(y_test, pipe1.predict_proba(X_test)[:, 1]))
report = classification_report(y_test, y_pred1)
print(report)

Accuracy: 0.82155
Precision: 0.3056193858366409
Recall: 0.8565573770491803
F1 Score: 0.4505003849114704
ROC-AUC: 0.9374423706041622
              precision    recall  f1-score   support

           0       0.98      0.82      0.89     18292
           1       0.31      0.86      0.45      1708

    accuracy                           0.82     20000
   macro avg       0.64      0.84      0.67     20000
weighted avg       0.93      0.82      0.86     20000



In [21]:
pipe2= Pipeline(steps=[
    ('trf1',trf1),
    ('smote',SMOTE(random_state=42)),
    ('rf',RandomForestClassifier(class_weight='balanced', random_state=42))
])
pipe2.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encode',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['gender', 'hypertension',
                                                   'heart_disease',
                                                   'smoking_history']),
                                                 ('scale', MinMaxScaler(),
                                                  ['age', 'bmi', 'HbA1c_level',
                                                   'blood_glucose_level'])])),
                ('smote', SMOTE(random_state=42)),
                ('rf',
                 RandomForestClassifier(class_weight='balanced',
                                        random_state=42))])

In [22]:
param_grid2 = {
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [None, 10, 20],
    'rf__min_samples_split': [2, 5],
    'rf__min_samples_leaf': [1, 2],
    'rf__class_weight': ['balanced', {0: 1, 1: 3}]
}

In [23]:
grid_search2 = GridSearchCV(
    pipe2,
    param_grid=param_grid2,
    scoring='f1',
    cv=5,
    n_jobs=-1,
    verbose=1
    )
grid_search2.fit(X_train, y_train)
print("Best Parameters:", grid_search2.best_params_)
print("Best Score:", grid_search2.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'rf__class_weight': 'balanced', 'rf__max_depth': None, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 200}
Best Score: 0.6965212774609884


In [24]:
model2 = grid_search2.best_estimator_
y_pred2 = model2.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred2))
print("Precision:", precision_score(y_test, y_pred2))
print("Recall:", recall_score(y_test, y_pred2))
print("F1 Score:", f1_score(y_test, y_pred2))
print("ROC-AUC:", roc_auc_score(y_test, pipe2.predict_proba(X_test)[:, 1]))
report = classification_report(y_test, y_pred2)
print(report)

Accuracy: 0.9464
Precision: 0.6702355460385439
Recall: 0.7330210772833724
F1 Score: 0.7002237136465325
ROC-AUC: 0.9548236268424124
              precision    recall  f1-score   support

           0       0.97      0.97      0.97     18292
           1       0.67      0.73      0.70      1708

    accuracy                           0.95     20000
   macro avg       0.82      0.85      0.84     20000
weighted avg       0.95      0.95      0.95     20000



In [25]:
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

In [26]:
pipe3= Pipeline(steps=[
    ('trf1',trf1),
    ('smote',SMOTE(random_state=42)),
    ('xgb',XGBClassifier(eval_metric='logloss'))
])
pipe3.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encode',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['gender', 'hypertension',
                                                   'heart_disease',
                                                   'smoking_history']),
                                                 ('scale', MinMaxScaler(),
                                                  ['age', 'bmi', 'HbA1c_level',
                                                   'blood_glucose_level'])])),
                ('smote', SMOTE(random_state=42)),
                ('xgb',
                 XGBClassifier(base_score=None...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [27]:
param_grid3 = {
    'xgb__n_estimators': [100, 300],
    'xgb__max_depth': [3, 5],
    'xgb__learning_rate': [0.1, 0.2],
    'xgb__subsample': [0.8, 1.0],
    'xgb__colsample_bytree': [0.8, 1.0]
}

In [28]:
grid_search3 = GridSearchCV(
    pipe3,
    param_grid=param_grid3,
    scoring='recall', 
    cv=3,
    n_jobs=-1,
    verbose=1
)
grid_search3.fit(X_train, y_train)
print("Best Parameters:", grid_search3.best_params_)
print("Best Score:", grid_search3.best_score_)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
Best Parameters: {'xgb__colsample_bytree': 1.0, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 100, 'xgb__subsample': 0.8}
Best Score: 0.8228798586572438


In [29]:
model3 = grid_search3.best_estimator_
y_pred3 = model3.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred3))
print("Precision:", precision_score(y_test, y_pred3))
print("Recall:", recall_score(y_test, y_pred3))
print("F1 Score:", f1_score(y_test, y_pred3))
print("ROC-AUC:", roc_auc_score(y_test, pipe3.predict_proba(X_test)[:, 1]))
report = classification_report(y_test, y_pred3)
print(report)

Accuracy: 0.9182
Precision: 0.5126138752627891
Recall: 0.8565573770491803
F1 Score: 0.64138535729943
ROC-AUC: 0.9710253448993712
              precision    recall  f1-score   support

           0       0.99      0.92      0.95     18292
           1       0.51      0.86      0.64      1708

    accuracy                           0.92     20000
   macro avg       0.75      0.89      0.80     20000
weighted avg       0.95      0.92      0.93     20000



In [37]:
from sklearn.ensemble import StackingClassifier
stacked = StackingClassifier(
    estimators=[
        ('lr_pipe1', pipe1),
        ('rf_pipe2', pipe2),
        ('xgb_pipe3',pipe3)
    ],
    final_estimator=LogisticRegression(max_iter=1000),
    n_jobs=-1,
)

stacked.fit(X_train, y_train)
y_pred = stacked.predict(X_test)
y_proba = stacked.predict_proba(X_test)[:, 1]
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))


Accuracy: 0.9599
Precision: 0.8306569343065694
Recall: 0.6662763466042154
F1 Score: 0.7394411955815464
ROC-AUC: 0.9682450186180878
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     18292
           1       0.83      0.67      0.74      1708

    accuracy                           0.96     20000
   macro avg       0.90      0.83      0.86     20000
weighted avg       0.96      0.96      0.96     20000



In [61]:
import pickle
with open('diabetes_model.pkl', 'wb') as file:
    pickle.dump(stacked, file)

In [63]:
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
with open("performance_report.txt", "w") as f:
    f.write("Model: Stacked Ensemble\n")
    f.write("Accuracy: {:.4f}\n".format(accuracy_score(y_test, y_pred)))
    f.write("Precision: {:.4f}\n".format(precision_score(y_test, y_pred)))
    f.write("Recall: {:.4f}\n".format(recall_score(y_test, y_pred)))
    f.write("F1 Score: {:.4f}\n".format(f1_score(y_test, y_pred)))
    f.write("ROC-AUC: {:.4f}\n".format(roc_auc_score(y_test, y_proba)))
    f.write("\nClassification Report:\n")
    f.write(report)
    f.write("\nConfusion Matrix:\n")
    f.write(np.array2string(cm))
